# Kanana RAG Fine-tuning Notebook

This notebook fine-tunes the Kanana 8B instruct model on RAG tasks using the Jecheon tourism dataset.

**Model:** kakaocorp/kanana-1.5-8b-instruct-2505

**Training Data Format:**
```
[Instruction]
당신은 제천시 관광 안내 전문가입니다.
제공된 여러 문서 중에서 질문과 관련된 문서를 찾아, 그 문서의 내용을 바탕으로 정확하고 친절하게 답변해주세요.

Information:
{content1}
Information:
{content2}
Question: {question}
```

In [ ]:
# Install required packages
!pip install transformers peft datasets wandb bitsandbytes accelerate

In [ ]:
# Check CUDA availability
import os
import torch

os.environ["NVIDIA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Login to Weights & Biases
import wandb

# Login to wandb (you'll be prompted for your API key)
wandb.login()

# Initialize wandb project
wandb.init(
    project="kanana-rag-finetuning",
    name="kanana-1.5-8b-instruct-rag",
    config={
        "model": "kakaocorp/kanana-1.5-8b-instruct-2505",
        "task": "RAG fine-tuning",
        "dataset": "Jecheon Tourism"
    }
)

In [ ]:
# Load base model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "kakaocorp/kanana-1.5-8b-instruct-2505"

print(f"Loading model: {model_name}")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

print("Model loaded successfully!")

In [ ]:
# Load RAG training data
import json
from datasets import Dataset

data_path = "/home/user/goodganglabs/data/processed/training_data.jsonl"

# Load JSONL data
data_list = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))

print(f"Loaded {len(data_list)} training examples")
print("\nFirst example:")
print(json.dumps(data_list[0], ensure_ascii=False, indent=2)[:500])

In [ ]:
# Convert to requested RAG format with instruction
def format_rag_data(example):
    """Convert RAG data to the requested format with instruction:
    [Instruction]
    당신은 제천시 관광 안내 전문가입니다...
    
    Information:
    {content1}
    Information:
    {content2}
    Question: {question}
    """
    instruction = """당신은 제천시 관광 안내 전문가입니다.
제공된 여러 문서 중에서 질문과 관련된 문서를 찾아, 그 문서의 내용을 바탕으로 정확하고 친절하게 답변해주세요.

답변 시 주의사항:
1. 관련 문서의 내용만을 바탕으로 답변하세요
2. 문서에 정보가 없으면 "제공된 정보에는 해당 내용이 없습니다"라고 답변하세요
3. 추측하거나 문서 외부 지식을 사용하지 마세요
4. 간결하고 이해하기 쉽게 답변하세요"""
    
    documents = example['documents']
    question = example['question']
    answer = example['answer']
    
    # Build information sections
    info_sections = []
    for doc in documents:
        info_sections.append(f"Information:\n{doc['content']}")
    
    # Combine: instruction + information sections + question
    prompt = instruction + "\n\n"
    prompt += "\n\n".join(info_sections)
    prompt += f"\n\nQuestion: {question}"
    
    return {
        "prompt": prompt,
        "answer": answer
    }

# Apply formatting
formatted_data = []
for example in data_list:
    formatted_data.append(format_rag_data(example))

print(f"Formatted {len(formatted_data)} examples")
print("\nExample formatted prompt:")
print(formatted_data[0]['prompt'][:500])
print(f"\nAnswer: {formatted_data[0]['answer']}")

In [ ]:
# Create training dataset with proper format
def formatting_prompts_func(examples):
    """Format prompts for training"""
    prompts = examples["prompt"]
    answers = examples["answer"]
    
    EOS_TOKEN = tokenizer.eos_token
    
    texts = []
    for prompt, answer in zip(prompts, answers):
        # Combine prompt and answer with EOS token
        text = f"{prompt}\n\nAnswer: {answer}{EOS_TOKEN}"
        texts.append(text)
    
    return {"text": texts}

# Create dataset
dataset = Dataset.from_list(formatted_data)
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"Dataset size: {len(dataset)}")
print(f"\nDataset features: {dataset.features}")
print(f"\nFirst training example:")
print(dataset[0]['text'][:500])

In [ ]:
# Tokenize dataset
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"], 
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_tensors="pt"
    )
    tokens["labels"] = tokens["input_ids"].clone()
    return tokens

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=["text", "prompt", "answer"]
)

print(f"Tokenized dataset size: {len(tokenized_dataset)}")
print(f"Features: {tokenized_dataset.features}")

In [ ]:
# Configure LoRA for parameter-efficient fine-tuning
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,                    # LoRA rank
    lora_alpha=32,          # LoRA alpha
    lora_dropout=0.1,       # Dropout probability
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj"
    ]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Log LoRA config to wandb
wandb.config.update({
    "lora_r": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
    "lora_dropout": lora_config.lora_dropout,
    "target_modules": lora_config.target_modules
})

In [ ]:
# Check GPU memory before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Log to wandb
wandb.config.update({
    "gpu_name": gpu_stats.name,
    "gpu_max_memory_gb": max_memory,
    "gpu_start_memory_gb": start_gpu_memory
})

In [ ]:
# Configure training arguments with wandb integration
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    # Output settings
    output_dir="./outputs/kanana-rag",
    overwrite_output_dir=True,
    
    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=10,
    
    # Optimization
    bf16=True,
    optim="adamw_torch",
    lr_scheduler_type="linear",
    
    # Logging
    logging_steps=5,
    logging_dir="./logs",
    report_to="wandb",  # Enable wandb reporting
    
    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    
    # Evaluation
    evaluation_strategy="no",  # No validation set in this example
    
    # Other
    seed=1234,
    data_seed=1234,
    remove_unused_columns=True,
)

print("Training arguments configured:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Total steps: ~{len(tokenized_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("Trainer initialized successfully!")

In [ ]:
# Start training
print("Starting training...")
print("=" * 50)

trainer_stats = trainer.train()

print("\n" + "=" * 50)
print("Training completed!")
print(f"Training loss: {trainer_stats.training_loss:.4f}")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f}s")

In [ ]:
# Check final GPU memory usage
final_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nFinal GPU memory: {final_gpu_memory} GB")
print(f"Peak memory used: {final_gpu_memory - start_gpu_memory} GB")

# Log final stats to wandb
wandb.log({
    "final_gpu_memory_gb": final_gpu_memory,
    "peak_memory_used_gb": final_gpu_memory - start_gpu_memory
})

In [ ]:
# Save the fine-tuned model
output_dir = "./outputs/kanana-rag-final"

print(f"Saving model to {output_dir}...")
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved successfully!")
print(f"\nModel location: {output_dir}")

In [ ]:
# Test inference with a sample
print("Testing inference...\n")

# Get a test example
test_prompt = formatted_data[0]['prompt']
expected_answer = formatted_data[0]['answer']

print("Test Prompt:")
print(test_prompt[:300])
print("\n...")

# Tokenize and generate
model.eval()
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nGenerated Response:")
print(generated_text[len(test_prompt):])

print("\nExpected Answer:")
print(expected_answer)

In [ ]:
# Create detailed examples table for wandb
examples_data = []

for i in range(min(5, len(test_samples))):
    examples_data.append([
        test_samples[i]['question'],
        test_samples[i]['answer'],
        predictions[i]['answer'],
        test_samples[i].get('question_type', 'unknown')
    ])

examples_table = wandb.Table(
    columns=["Question", "Reference Answer", "Generated Answer", "Question Type"],
    data=examples_data
)

wandb.log({"eval/detailed_examples": examples_table})

print("✓ Example predictions logged to WandB")
print("\nView your results at: https://wandb.ai")

In [ ]:
# Run generation-only evaluation (no docs needed!)
print("\n" + "=" * 60)
print("Running generation-only evaluation...")
print("=" * 60 + "\n")

# Evaluate - only needs 'answer' field in predictions and dataset
results = evaluator.evaluate_generation_only(
    dataset=test_samples,
    model_predictions=predictions
)

# Display results
print(evaluator.format_results_generation_only(results))

# Log to wandb
wandb.log({
    "eval/rouge1": results['rouge1'],
    "eval/rouge2": results['rouge2'],
    "eval/rougeL": results['rougeL'],
    "eval/bert_f1": results['bert_f1'],
    "eval/bert_precision": results['bert_precision'],
    "eval/bert_recall": results['bert_recall'],
    "eval/exact_match": results['exact_match'],
    "eval/num_samples": results['num_samples']
})

print("\n✓ Results logged to WandB")

In [ ]:
# Generate predictions on test set
print("Generating predictions on test set...")
print("=" * 60)

# Use first 30 samples for evaluation (or load separate test set)
test_samples = data_list[:30]  
predictions = []

model.eval()
with torch.no_grad():
    for i, sample in enumerate(test_samples):
        # Format prompt
        formatted = format_rag_data(sample)
        prompt = formatted['prompt']
        
        # Generate answer
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # Extract generated text
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = generated[len(prompt):].strip()
        
        # Remove "Answer:" prefix if present
        if answer.startswith("Answer:"):
            answer = answer[7:].strip()
        
        predictions.append({
            'answer': answer
        })
        
        if (i + 1) % 10 == 0:
            print(f"  Generated {i + 1}/{len(test_samples)} predictions...")

print(f"\n✓ Generated {len(predictions)} predictions")

# Show example
print("\n" + "=" * 60)
print("Example prediction:")
print("=" * 60)
print(f"Question: {test_samples[0]['question']}")
print(f"\nGenerated: {predictions[0]['answer'][:200]}...")
print(f"\nReference: {test_samples[0]['answer'][:200]}...")

In [ ]:
# Import evaluation metrics
import sys
sys.path.insert(0, '/home/user/goodganglabs')

from src.evaluation.metrics import create_evaluator

# Create evaluator (no k_values needed for generation-only)
evaluator = create_evaluator()

print("✓ Evaluator initialized")
print("  Available metrics: ROUGE, BERTScore, Exact Match")

In [ ]:
# Install evaluation dependencies if needed
!pip install -q rouge-score bert-score

## Evaluation with Generation-Only Metrics

Now let's evaluate the fine-tuned model using generation-only metrics (ROUGE, BERTScore, Exact Match).

**No document retrieval metrics needed** - we only compare generated answers vs ground truth.

In [ ]:
# Close wandb run
wandb.finish()
print("WandB run finished!")

## Summary

This notebook:
1. ✅ Loads Kanana 1.5 8B instruct model (kakaocorp/kanana-1.5-8b-instruct-2505)
2. ✅ Formats RAG training data with instruction and information format:
   ```
   [Instruction: 제천시 관광 안내 전문가 역할]
   
   Information:
   {content1}
   Information:
   {content2}
   Question: {question}
   
   Answer: {answer}
   ```
3. ✅ Applies LoRA for efficient fine-tuning
4. ✅ Tracks training with Weights & Biases
5. ✅ Saves the fine-tuned model
6. ✅ Tests inference on sample data

### Key Features:
- System instruction guides the model to be a Jecheon tourism expert
- Model learns to find relevant documents and answer based on them
- Model learns to say "no information available" when appropriate
- Training format prevents hallucination

### Next Steps:
- Run full evaluation on test set
- Compare baseline vs fine-tuned performance
- Upload model to Hugging Face Hub
- Generate report with metrics and examples